In [1]:
!pip install -r ../../../requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.118.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import pandas as pd
import openai
from utils_notebook import *
import utils_notebook as utils
import constants
from sklearn.metrics.pairwise import cosine_similarity


c:\Users\ASUS\Desktop\Stuff\thesius.ai\thesius.ai\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\Desktop\Stuff\thesius.ai\thesius.ai\backend\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
openai.api_key = constants.OPENAI_API_KEY

### Query S2 paper index

In [9]:
query = "How RAG works and how it impacts AI ?"

# json to pandas dataframe
search_results = search(preprocess_query(query))

if search_results['total']==0:
    print('No results found - Try another query')
else:
    df = pd.DataFrame(search_results['data']).dropna()

In [10]:
df.head()

,paperId,title,abstract,venue,year
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024


### Re-ranking using SPECTER

In [11]:
# merge columns title and abstract into a string separated by tokenizer.sep_token and store it in a list

df['title_abs'] = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in df.to_dict('records')]
df['n_tokens'] = df.title_abs.apply(lambda x: len(tokenizer.encode(x)))
df['title_abs'][2]

"AI-Driven Marketplaces and Price Prediction Tools for Rag Pickers: Enhancing Economic Opportunities in Africa's Circular Economy[SEP]This paper aims at identifying and analyzing the use of the AI solutions in enhancing the economic status of rag pickers within Africa’s circular economy sector and their access to markets. In this study, we explore how effectively the employment of AI technologies advances the welfare of the weaver people by identifying new potential methods of income generation and skill development in waste management techniques. Our research adopts a case study approach, examining the potential of two key AI applications: The resources include a marketplace that helps rag pickers find buyers for recyclable materials and price forecasting applications to aid selling. The role of such technologies in enhancing the efficiency of materials exchange, increasing or decreasing income, and improving rag pickers’ understanding of the market is discussed. The study entailed th

In [12]:
# get embeddings for each document and query

doc_embeddings = get_specter_embeddings(list(df['title_abs']))
query_embeddings = get_specter_embeddings(advanced_preprocess_query(query))

df['specter_embeddings'] = list(doc_embeddings)
df.head()

[+] processed query:  Query expanded How RetrievalAugmented Generation RAG works impacts Artificial Intelligence AI


,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952..."
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021,Impact of Artificial Intelligence in the field...,121,"[0.24786988, 0.2989162, -0.72270244, 0.2948508..."
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023,AI-Driven Marketplaces and Price Prediction To...,324,"[0.0011445875, 0.22520094, -0.6562357, -0.0665..."
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023,Corporate Governance in the Digital Age: A Com...,275,"[0.3889911, -0.013424439, -0.86219573, 0.02648..."
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024,Empirically Understanding the Potential Impact...,306,"[0.2136501, 0.27276045, -0.93551767, 0.2673642..."


In [9]:
# find the cosine similarity between the query and the documents
df['similarity'] = cosine_similarity(query_embeddings, doc_embeddings).flatten()

# sort the dataframe by similarity
df.sort_values(by='similarity', ascending=False, inplace=True)
df.head()

,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings,similarity
16,77179e5ff669452b9bea479a4236a6e2009ee422,The Power of Noise: Redefining Retrieval for R...,Retrieval-Augmented Generation (RAG) has recen...,Annual International ACM SIGIR Conference on R...,2024,The Power of Noise: Redefining Retrieval for R...,290,"[0.046463024, 0.43938, -0.7837731, -0.26589015...",0.923377
12,917cef7bf9841044af50f4bdd75db057aba1aed9,RAGProbe: An Automated Approach for Evaluating...,Retrieval Augmented Generation (RAG) is increa...,arXiv.org,2024,RAGProbe: An Automated Approach for Evaluating...,312,"[0.27762347, 0.38148606, -0.8177407, -0.270688...",0.902334
10,dcbcba4cf01b36e499060a7d4e9a00e808f4daf9,ML Based Solutions for Greenhouse Gas Emission...,This literature review will serve as the basis...,International Journal on Perceptive and Cognit...,2023,ML Based Solutions for Greenhouse Gas Emission...,226,"[0.22443536, 0.23313893, -0.8218236, 0.1431330...",0.895758
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952...",0.893167
5,e9ad33f3579d61b4a8ad684edd03cb10df6a27d0,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,The paper focuses on the science fiction novel...,SAKHAROV READINGS 2022: ENVIRONMENTAL PROBLEMS...,2022,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,179,"[-0.11198905, 0.31442636, -0.8300437, -0.09335...",0.893007


## Get answer from GPT-3

In [13]:
# Define a function to generate a summary of each abstract
def summarize_abstract(abstract):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Summarize the following abstract:\n\n{abstract}"}],
        max_tokens=50,
        temperature=0.7
    )
    summary = response.choices[0].message.content.strip()
    return summary



# Apply the functions to create the abstract_summary and relevance columns
df['abstract_summary'] = df['abstract'].apply(summarize_abstract)


# Display the updated DataFrame
df.head()



,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings,abstract_summary
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952...",The article discusses the profound impact of A...
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021,Impact of Artificial Intelligence in the field...,121,"[0.24786988, 0.2989162, -0.72270244, 0.2948508...",The abstract discusses the transformative pote...
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023,AI-Driven Marketplaces and Price Prediction To...,324,"[0.0011445875, 0.22520094, -0.6562357, -0.0665...",This paper explores the use of AI solutions to...
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023,Corporate Governance in the Digital Age: A Com...,275,"[0.3889911, -0.013424439, -0.86219573, 0.02648...",This research explores the impact of disruptiv...
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024,Empirically Understanding the Potential Impact...,306,"[0.2136501, 0.27276045, -0.93551767, 0.2673642...","In this study, researchers explore how Artific..."


In [18]:
df
df_now = df.drop(columns=['title_abs', 'n_tokens', 'specter_embeddings'])
df_now

,paperId,title,abstract,venue,year,abstract_summary
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,The article discusses the profound impact of A...
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021,The abstract discusses the transformative pote...
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023,This paper explores the use of AI solutions to...
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023,This research explores the impact of disruptiv...
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024,"In this study, researchers explore how Artific..."
5,e9ad33f3579d61b4a8ad684edd03cb10df6a27d0,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,The paper focuses on the science fiction novel...,SAKHAROV READINGS 2022: ENVIRONMENTAL PROBLEMS...,2022,"The paper examines the themes of posthumanism,..."
6,9cd1d535c50cc4ebba5db00c686547a2ee542e26,Imagine a More Ethical AI: Using Stories to De...,Artificial intelligence (AI) tools and technol...,Research in Equitable and Sustained Participat...,2021,The abstract discusses the increasing prevalen...
7,947eff59ac983cf31339742da23a26b53406bf3b,The copyright protection of AI-generated works...,Who is the author of a work generated by AI? C...,Juridical Tribune,2023,The abstract discusses the issue of authorship...
8,0ae9ccbeec8befa049edb1576df53b853cc4ee76,How AI Affects Our Understanding of Musical Wo...,Applying Artificial Intelligence (AI) algorith...,IEEE International Joint Conference on Neural ...,2023,This abstract discusses how the use of Artific...
9,28c2ea859996435501eea60a5b5baf383a49972a,AUTHORSHIP OF AI-GENERATED WORKS IN ARTISTIC D...,The article analyzes the question of authorshi...,Grail of Science,2023,The article explores the impact of AI on the a...


In [12]:
answer_question(df, question=query, debug=False)


'Retrieval-Augmented Generation (RAG) is a method that extends beyond the pre-trained knowledge of Large Language Models (LLMs) by augmenting the original prompt with relevant passages or documents retrieved by an Information Retrieval (IR) system. RAG systems are increasingly important for Generative AI solutions, especially in scenarios where knowledge is constantly updated and cannot be memorized in the LLM. The retrieval component of RAG systems, whether dense or sparse, plays a crucial role in enhancing the effectiveness of LLMs. One key finding is that adding random documents in the prompt can improve the LLM accuracy by up to 35%. Additionally, the type of passages that IR systems within a RAG solution retrieve, their relevance to the query'